### 二叉树结点类

In [3]:
class BinTNode:
    def __init__(self, dat, left = None, right = None):
        self.data = dat
        self.left = left
        self.right = right

In [4]:
# 一棵包含了三个结点的二叉树，变量t引着树根结点
t = BinTNode(1, BinTNode(2), BinTNode(3))

In [5]:
def count_BinTNode(t):
    if t is None:
        return 0
    else:
        return 1 + count_BinTNode(t, left) \
               + count_BinTNode(t, right)

# 假设结点中保存数值，求这种二叉树里的所有数值和：
def sum_BinTNode(t):
    if t is None:
        return 0
    else:
        return t.data + sum_BinTNode(t.left) \
               + sum_BinTNodes(t, right)

### 遍历算法
如何针对链接的二叉树结点构成的二叉树，实现遍历二叉树的函数？

In [6]:
def preorder(t, proc): # proc是具体的结点数据操作
    assert (isinstance(t, BinTNode))
    if t is None:
        return
    proc(t.data)
    preorder(t.left)
    preorder(t.right)

# 链接二叉树的可视化打印    
def print_BinTNodes(t):
    if t is None:
        print("^", end = "") # 空树输出^
        return
    print("(" + str(t.data), end = "")
    print_BinTNodes(t.left)
    print_BinTNodes(t.right)
    print(")", end = "")

In [7]:
t = BinTNode(1, BinTNode(2, BinTNode(5)), BinTNode(3))
print_BinTNodes(t)

(1(2(5^^)^)(3^^))

### 宽度优先遍历

In [8]:
from SQueue import *
def levelorder(t, proc):
    qu = SQueue()
    qu.enqueue(t)
    while not qu.is_empty():
        t = qu.dequeue()
        if t is None: # 弹出的树为空树则直接跳过
            continue
        qu.enqueue(t.left)
        qu.enqueue(t.right)
        proc(t.data)

In [27]:
# 一个测试，查看该操作可以做啥
t = BinTNode(1,
             BinTNode(2,
                      BinTNode(5,
                               BinTNode(8),BinTNode(9,
                                                    None, BinTNode(11))),
                      BinTNode(6)
                     ),
             BinTNode(3, 
                      BinTNode(7,BinTNode(4),BinTNode(10)))
            )

print("树形结构是",end = ": \n")
print_BinTNodes(t)
print("\n宽度优先遍历结果: ")
levelorder(t,lambda x: print(x, end = " "))

树形结构是: 
(1(2(5(8^^)(9^(11^^)))(6^^))(3(7(4^^)(10^^))^))
宽度优先遍历结果: 
1 2 3 5 6 7 8 9 4 10 11 

### 非递归的先根序遍历函数

In [19]:
def preorder_nonrec(t, proc):
    s = SStack()
    while t is not None or not s.is_empty():
        while t is not None: # 沿左分支下行
            proc(t.data) # 先根序处理根数据
            s.push(t.right) # 右分支入栈
            t = t.left
        t = s.pop() # 遇到空树，回溯

In [28]:
preorder_nonrec(t, lambda x:print(x, end = " "))

1 2 5 8 9 11 6 3 7 4 10 

### 非递归的后根序遍历算法

In [12]:
def postorder_nonrec(t, proc):
    s = SStack()
    while t is not None or not s.is_empty():
        while t is not None:
            s.push(t)
            # 这个条件表达式的意义是能左就左，否则往右一步
            t = t.left if t.left is not None else t.right
        # 后进先出，最后肯定是
        t = s.pop()
        proc(t.data)
        if not s.is_empty() and s.top().left == t:
            t = s.top().right # 栈不空，且当前结点是栈顶的左子结点
        else:
            t = None # 没有右子树或者右子树遍历完毕，强迫退栈

In [29]:
postorder_nonrec(t, lambda x: print(x, end = " "))

8 11 9 5 6 2 4 10 7 3 1 

这个算法想法还是蛮复杂的。整理一下主要是：
1. 首先顺着根节点找到最左边的叶结点，并一路把元素压入栈，终止
2. 如果（终止时）被访问的是左结点，那么转为它的兄弟结点继续
3. 如果终止的时候是右结点，则设t为None，这样就会pop出上一层的结点，进行下一次循环

## 二叉树类
基于结点构造的二叉树有递归的结构，但是None表示空树，不是BinTNode，所以不是一种良好的封装的抽象数据类型；

要解决这类问题，最好的方法就是定义一个二叉树类，用BinTNode结点链接而成的树形结构作为其内部表示。

In [40]:
class BinTree:
    def __init__(self):
        self._root = None
        
    def is_empty(self):
        return self._root is None
    
    def root(self):
        return self._root
    
    def leftchild(self):
        return self._root.left
    
    def rightchild(self):
        return self._root.right
    
    def set_root(self, rootnode):
        self._root = rootnode
    
    def set_left(self, leftchild):
        self._root.left = leftchild
    
    def set_right(self, rightchild):
        self._root.right = rightchild
    
    # 一个元素迭代器（按照先根序）
    def preorder_elements(self):
        t, s = self._root, SStack()
        while t is not None or not s.is_empty():
            while t is not None:
                s.push(t.right)
                yield (t.data)
                t = t.left
            t = s.pop()

In [61]:
a = BinTree()

a.set_root(BinTNode("root"))
a.set_left(BinTNode("left"))
a.set_right(t)

In [62]:
for i in a.preorder_elements():
    print (i,end = "->")

root->left->1->2->5->8->9->11->6->3->7->4->10->